# QA Feat1 - Chocolate 

Check z-stats should be present  
Check errors in logs   
Look through logs   

In [ ]:
import glob, os, subprocess
import pandas as pd
import subprocess

QA_dict = {}
sessions = ["ses-1", "ses-2", "ses-3", "ses-4"]
deriv_path='/projects/niblab/bids_projects/Experiments/ChocoData/derivatives'
SUBJECTS = sorted(glob.glob(os.path.join(deriv_path, "sub-*")))

ses1_count = len(glob.glob("%s/sub-*/ses-1"%(deriv_path)))
ses2_count = len(glob.glob("%s/sub-*/ses-2"%(deriv_path)))
ses3_count = len(glob.glob("%s/sub-*/ses-3"%(deriv_path)))
ses4_count = len(glob.glob("%s/sub-*/ses-4"%(deriv_path)))
    

# Check for error in report log
errors = ['error', 'ERROR', 'Error']
for ses in sessions:
    if ses not in QA_dict:
        QA_dict[ses] = {}
    for error in errors:
        error_cmd = "grep '%s' %s/sub-*/%s/func/Analysis/feat1/task*/report_log.html"%(error, deriv_path, ses)
        #print(error_cmd)
        try:
            output = subprocess.check_output(error_cmd, shell=True)
            # IF ERROR OUTPUT WRITE TO FILE ....
        # IF NOTHING FOUND EXCEPT ERROR AND PASS
        except subprocess.CalledProcessError as e:
            if e.returncode > 1:
                raise
    # Check for existence of zstats 
    # Expecting 2 milkshake runs for each subject - therefore 2 zstats per subject
    subs = sorted(glob.glob("%s/sub-*"%(deriv_path)))
    for sub_dir in subs:
        #print(sub_dir)
        sub_id = sub_dir.split("/")[-1]

        if os.path.exists("%s/%s"%(sub_dir, ses)):
            zstats = glob.glob("%s/%s/func/Analysis/feat1/task*/stats/zstat1.nii.gz"%(sub_dir, ses))
            if len(zstats) != 2:
                if len(zstats) == 0:
                    status = None
                else:
                    status = zstats
                if sub_id not in QA_dict[ses]:
                    QA_dict[ses][sub_id] = {"FILES_FOUND" : status}
                

    # -count 
missing =[]
single_runs = []
for ses in QA_dict:
    for sub_id in QA_dict[ses]:
        if QA_dict[ses][sub_id]["FILES_FOUND"] == None:
            missing.append("%s_%s"%(ses,sub_id))
        else:
            single_runs.append("%s_%s"%(ses,sub_id))
            
print("Session 1 Count > ", ses1_count) 
print("Session 1 Count > ", ses2_count) 
print("Session 3 Count > ", ses3_count) 
print("Session 4 Count > ", ses4_count) 

info_dict = {
    "SES1_CT": ses1_count,
    "SES2_CT": ses2_count,
    "SES3_CT": ses3_count,
    "SES4_CT": ses4_count,
    "SES1_SINGLE": [i for i in single_runs if "ses-1" in i],
    "SES2_SINGLE": [i for i in single_runs if "ses-2" in i],
    "SES3_SINGLE": [i for i in single_runs if "ses-3" in i],
    "SES4_SINGLE": [i for i in single_runs if "ses-4" in i],
    "MISSING": missing 
    
}

df = pd.DataFrame.from_dict(info_dict, orient='index')

Session 1 Count >  115  
Session 1 Count >  115  
Session 3 Count >  104  
Session 4 Count >  93   


In [ ]:
# Session 1 subject with single run only
df.iloc[4,0]

['ses-1_sub-015',
 'ses-1_sub-032',
 'ses-1_sub-033',
 'ses-1_sub-059',
 'ses-1_sub-081',
 'ses-1_sub-088']

In [ ]:
# Session 2 subject with single run only
df.iloc[5,0]

['ses-2_sub-045',
 'ses-2_sub-078',
 'ses-2_sub-081',
 'ses-2_sub-088',
 'ses-2_sub-151',
 'ses-2_sub-152']

In [ ]:
# Session 3 subject with single run only
df.iloc[6,0]

['ses-3_sub-013',
 'ses-3_sub-032',
 'ses-3_sub-048',
 'ses-3_sub-094',
 'ses-3_sub-097',
 'ses-3_sub-118',
 'ses-3_sub-132']

In [ ]:
# Session 4 subject with single run only
df.iloc[7,0]

 ['ses-4_sub-092', 'ses-4_sub-101', 'ses-4_sub-103', 'ses-4_sub-120']

#### Make HTML file

In [ ]:
# Code below extended from Jeannette Mumford Brain stats - 
import os
import glob
import pdfkit
import pandas as pd
# We will start with the registration png files
outfile = "/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/feat1_ana.html"
#os.system("rm %s"%(outfile))
 
sessions = ["ses-1", "ses-2", "ses-3", "ses-4"]
dataframes = []

#f = open(outfile, "w")
for ses in sessions:
    all_feats = glob.glob('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/sub-*/%s/func/Analysis/feat1/task*'%ses)
    ses_dict = {}
    
    for file in list(all_feats):
        subject = file.split("/")[7]
        milk_id = file.split("/")[-1].split(".")[0].split("-")[1]
        filename = "%s_%s"%(subject, milk_id)
        if subject not in ses_dict:
            ses_dict[subject] = {}
        

        design_img = "%s/design.png"%(file)
        design_cov_img = "%s/design_cov.png"%(file)
        func2highres_img =  "%s/reg/example_func2highres.png"%(file)
        func2standard_img = "%s/reg/example_func2standard.png"%(file)
        highres2standard_img = "%s/reg/highres2standard.png"%(file)
        if os.path.exists(design_img) and milk_id not in ses_dict[subject]:
            ses_dict[subject][milk_id] = [design_img, design_cov_img, func2highres_img, func2standard_img, highres2standard_img]
            
        
        df = pd.DataFrame.from_dict({(i,j): ses_dict[i][j] 
                           for i in ses_dict.keys() 
                           for j in ses_dict[i].keys()},
                       orient='index')
        data_frames.append({'df':df})
    #os.system("cp %s/design.png files/%s_design.png"%(file, filename ))
    #os.system("cp %s/design_cov.png files/%s_design.png"%(file, filename))
    #os.system("cp %s/reg/example_func2highres.png files/%s_design.png"%(file, filename))
    #os.system("cp %s/reg/example_func2standard.png files/%s_design.png"%(file, filename))
    #os.system("cp %s/reg/highres2standard.png files/%s_design.png"%(file, filename))
    

    ## Building a PDF 
    # Create a Document 
    const PDFDocument = req
    const file_bio = "Subject: %s | Run: %s  \n"%(subject, milk_id)
    
    
f.close()

Regressor looks like waant.  
Flat regressor - mispecified the TR length/something wrong with offset file 
Colinearity pics:  
topleft check for pairwise colinearity  
Registration summary:  
Top : bold(brain) outline(MNI) | Bottom : vice versa .   
Bold to subject structural, subject structural to MNI template  


Reference:
- https://www.dropbox.com/s/rzcqb3nwvc01o1e/4_QA_all_lev1s.py?dl=0
- https://www.youtube.com/watch?v=kW1xgGs4NUU&list=PLB2iAtgpI4YHlH4sno3i3CUjCofI38a-3&index=13